In [144]:
# Hyper parameter tuning using hyperopt
# Written by Shengsi Li
# 20200911
# 'cext_stat_cde_h1e', 'cext_stat_cde_h1r' are encoded. The original model is 'cext_stat_cde_h1'
# Instead of 117+1(dep), use 116+1(dep)

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Standard plotly imports
#import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
#import cufflinks
#import cufflinks as cf
import plotly.figure_factory as ff

# Using plotly + cufflinks in offline mode
init_notebook_mode(connected=True)
#cufflinks.go_offline(connected=True)

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import xgboost as xgb

## Hyperopt modules
import hyperopt
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING
from functools import partial

import os
import gc


In [ ]:
# variable list
#var_list = pd.read_csv('/collectionproject-shared/CMAModeling/VaR_Rebuild_2020/DATA_DataPreparation/var_by_sample/var_list_0813.R', sep=",",index_col=None,header = None)
#var_list = pd.read_csv('/home/sli002/GH_VaR2.0_2020_MRM/prelim_models/xgb/Hyperopt/model_fit_full_sample_true117_var_list_MRM/true117_var_list.csv',header = None)

In [15]:
# data input (half sample)
#dev_cext_stat_cde_h1 = pd.read_csv('/collectionproject-shared/CMAModeling/VaR_Rebuild_2020/DATA_DataPreparation/var_by_sample/var_dev_fnl.txt', sep=",",usecols=['CEXT_STAT_CDE_H1'])
#val_cext_stat_cde_h1 = pd.read_csv('/collectionproject-shared/CMAModeling/VaR_Rebuild_2020/DATA_DataPreparation/var_by_sample/var_val_fnl.txt', sep=",",usecols=['CEXT_STAT_CDE_H1'])

In [24]:
# not run
# data input (full sample)
#dev_116 = pd.read_csv('/home/sli002/GH_VaR2.0_2020_MRM/prelim_models/xgb/Hyperopt/model_fit_full_sample_117_var_list_MRM/dev_117.csv',sep=",")
#val_116 = pd.read_csv('/home/sli002/GH_VaR2.0_2020_MRM/prelim_models/xgb/Hyperopt/model_fit_full_sample_117_var_list_MRM/val_117.csv',sep=",")

In [27]:
# combine
#dev_true117 = pd.concat([dev_116, dev_cext_stat_cde_h1], axis=1)
#val_true117 = pd.concat([val_116, val_cext_stat_cde_h1], axis=1)

In [30]:
# not run val_117 = val[var_list[0]]

In [58]:
# not run val_117.shape

(752195, 116)

In [37]:
#dev_true117.to_csv(r'/home/sli002/GH_VaR2.0_2020_MRM/prelim_models/xgb/Hyperopt/model_fit_full_sample_true117_var_list_MRM/dev_true117.csv', sep=',', index = False)
#val_true117.to_csv(r'/home/sli002/GH_VaR2.0_2020_MRM/prelim_models/xgb/Hyperopt/model_fit_full_sample_true117_var_list_MRM/val_true117.csv', sep=',', index = False)

In [ ]:
# not run val = pd.read_csv('/collectionproject-shared/CMAModeling/VaR_Rebuild_2020/DATA_DataPreparation/var_by_sample/var_val_fnl.txt', sep=",")

In [35]:
#dev_true117.columns = map(str.lower, dev_true117.columns) # column to lower case
#val_true117.columns = map(str.lower, val_true117.columns) # column to lower case

In [43]:
#dev_true117["cext_stat_cde_h1"].value_counts()

R    606276
F     84935
E     54818
C      6480
A      1085
Name: cext_stat_cde_h1, dtype: int64

In [2]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 

In [3]:
# load data
x_train = pd.read_csv('/home/sli002/GH_VaR2.0_2020_MRM/prelim_models/xgb/Hyperopt/model_fit_full_sample_true117_var_list_MRM/dev_true117.csv', sep=',')
x_test = pd.read_csv('/home/sli002/GH_VaR2.0_2020_MRM/prelim_models/xgb/Hyperopt/model_fit_full_sample_true117_var_list_MRM/val_true117.csv', sep=',')
y_train = x_train['mapa_bad']
y_test = x_test['mapa_bad']

In [4]:
# Label Encoding
x_train["cext_stat_cde_h1"] = label_encoder.fit_transform(x_train['cext_stat_cde_h1']) 
x_test["cext_stat_cde_h1"] = label_encoder.fit_transform(x_test['cext_stat_cde_h1']) 

In [5]:
x_train['cext_stat_cde_h1'].value_counts()

4    606276
3     84935
2     54818
1      6480
0      1085
Name: cext_stat_cde_h1, dtype: int64

In [6]:
x_test.shape

(752195, 117)

In [7]:
x_train.shape

(753594, 117)

In [8]:
y_train.shape

(753594,)

In [9]:
y_test.shape

(752195,)

In [10]:
space={'max_depth': hp.quniform("max_depth", 3, 12, 1),
        'gamma': hp.uniform ('gamma', 0.0000001,3),
        'reg_alpha' : hp.quniform('reg_alpha', 0,10,1),
        'reg_lambda' : hp.quniform('reg_lambda', 0,10,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.3,1),
        'min_child_weight' : hp.quniform('min_child_weight', 100, 2000, 1),
        'n_estimators': 200,
       'learning_rate': hp.uniform('learning_rate', 0.001, 0.2),
       'subsample': hp.uniform ('subsample', 0.3,1),
       'col_sample_rate' : hp.uniform('col_sample_rate',0.3,1)
       }

In [13]:
# Classifier:
def hyperparameter_tuning(space):
    model = xgb.XGBClassifier(n_estimators =space['n_estimators'],
                              max_depth = int(space['max_depth']),
                              gamma = space['gamma'],
                              reg_alpha = int(space['reg_alpha']),
                              min_child_weight=int(space['min_child_weight']),
                              learning_rate=space['learning_rate'],
                              col_sample_rate=space['col_sample_rate'],
                              colsample_bytree=space['colsample_bytree'])
    evaluation = [( x_train, y_train), ( x_test, y_test)]
    
    model.fit(x_train, y_train,
            eval_set=evaluation, eval_metric="rmse",
            early_stopping_rounds=20,verbose=True)

    pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    #change the metric if you like
    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}


In [14]:
trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

print (best)

[0]	validation_0-rmse:0.49292	validation_1-rmse:0.49292

Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.


Will train until validation_1-rmse hasn't improved in 20 rounds.

[1]	validation_0-rmse:0.48595	validation_1-rmse:0.48595

[2]	validation_0-rmse:0.47912	validation_1-rmse:0.47912

[3]	validation_0-rmse:0.47239	validation_1-rmse:0.47239

[4]	validation_0-rmse:0.46835	validation_1-rmse:0.46835

[5]	validation_0-rmse:0.46175	validation_1-rmse:0.46175

[6]	validation_0-rmse:0.45781	validation_1-rmse:0.45781

[7]	validation_0-rmse:0.45138	validation_1-rmse:0.45137

[8]	validation_0-rmse:0.44502	validation_1-rmse:0.44501

[9]	validation_0-rmse:0.43875	validation_1-rmse:0.43875

[10]	validation_0-rmse:0.43509	validation_1-rmse:0.43508

[11]	validation_0-rmse:0.42896	validation_1-rmse:0.42896

[12]	validation_0-rmse:0.42293	validation_1-rmse:0.42292

[13]	validation_0-rmse:0.41698	validation_1-rmse:0.41698

[14]	validation_0-rmse:0.41111	valida